# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [2]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [3]:
def nondense_model():
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

def dense_model_takelog():    
    inp = tf.keras.layers.Input(shape=(1,3))
    
    d1 = tf.keras.layers.Dense(1024, activation="sigmoid")(inp)
    d2 = tf.keras.layers.Dense(512, activation="sigmoid")(d1)
    d3 = tf.keras.layers.Dense(256, activation="sigmoid")(d2)
    d4 = tf.keras.layers.Dense(128, activation="sigmoid")(d3)
    d5 = tf.keras.layers.Dense(64, activation="sigmoid")(d4)
    
    o = tf.keras.layers.Dense(1)(d5)
    
    model = Model(inp,o)
    
    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model

def nondense_model_log():
    # Input layer of 3 neurons 
    inp = tf.keras.layers.Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128, activation="sigmoid")(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64, activation="sigmoid")(d2_out_p1)
    d4_out = Dense(64, activation="sigmoid")(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

### Datasets

In [4]:
num_examples = 20000
rstart = 1
rend = 10000

# MULT
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend)
mult_setX = mult_setX.reshape(num_examples, 1, 3)

### Build ModelComparator

In [5]:
mc = ModelComparator()

# mc.add_model("nondense_model", nondense_model)
# mc.add_model("dense_model", dense_model_5L)
mc.add_model("dense_model-log", dense_model_takelog)
mc.add_model("nondense_model-log", nondense_model_log)

mc.add_training_set("multiplication", (mult_setX, mult_setY))

### Train Models

In [ ]:
num_epochs = 1000
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

MODEL dense_model-log TRAINING ON DATASET multiplication
Epoch 1/1000
1/1 [==============================] - 0s 23ms/step - loss: 1101934324875264.0000 - accuracy: 0.0000e+00
Epoch 2/1000
1/1 [==============================] - 0s 2ms/step - loss: 1101934324875264.0000 - accuracy: 0.0000e+00
Epoch 3/1000
1/1 [==============================] - 0s 3ms/step - loss: 1101934056439808.0000 - accuracy: 0.0000e+00
Epoch 4/1000
1/1 [==============================] - 0s 997us/step - loss: 1101934324875264.0000 - accuracy: 0.0000e+00
Epoch 5/1000
1/1 [==============================] - 0s 2ms/step - loss: 1101934056439808.0000 - accuracy: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 0s/step - loss: 1101934190657536.0000 - accuracy: 0.0000e+00
Epoch 7/1000
1/1 [==============================] - 0s 998us/step - loss: 1101933989330944.0000 - accuracy: 0.0000e+00
Epoch 8/1000
1/1 [==============================] - 0s 2ms/step - loss: 1101934190657536.0000 - accuracy: 0.0000e+00
Epo

1/1 [==============================] - 0s 13ms/step - loss: 1101934056439808.0000 - accuracy: 0.0000e+00
Epoch 71/1000
1/1 [==============================] - 0s 5ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 72/1000
1/1 [==============================] - 0s 22ms/step - loss: 1101934056439808.0000 - accuracy: 0.0000e+00
Epoch 73/1000
1/1 [==============================] - 0s 8ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 74/1000
1/1 [==============================] - 0s 3ms/step - loss: 1101933989330944.0000 - accuracy: 0.0000e+00
Epoch 75/1000
1/1 [==============================] - 0s 2ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 76/1000
1/1 [==============================] - 0s 12ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 77/1000
1/1 [==============================] - 0s 9ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 78/1000
1/1 [==============================] - 0s 16ms/step -

1/1 [==============================] - 0s 4ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 140/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933720895488.0000 - accuracy: 0.0000e+00
Epoch 141/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101933855113216.0000 - accuracy: 0.0000e+00
Epoch 142/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933653786624.0000 - accuracy: 0.0000e+00
Epoch 143/1000
1/1 [==============================] - 0s 9ms/step - loss: 1101933653786624.0000 - accuracy: 0.0000e+00
Epoch 144/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933720895488.0000 - accuracy: 0.0000e+00
Epoch 145/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933653786624.0000 - accuracy: 0.0000e+00
Epoch 146/1000
1/1 [==============================] - 0s 3ms/step - loss: 1101933720895488.0000 - accuracy: 0.0000e+00
Epoch 147/1000
1/1 [==============================] - 0s 0s/step - 

1/1 [==============================] - 0s 16ms/step - loss: 1101933385351168.0000 - accuracy: 0.0000e+00
Epoch 209/1000
1/1 [==============================] - 0s 18ms/step - loss: 1101933519568896.0000 - accuracy: 0.0000e+00
Epoch 210/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933385351168.0000 - accuracy: 0.0000e+00
Epoch 211/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101933519568896.0000 - accuracy: 0.0000e+00
Epoch 212/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101933653786624.0000 - accuracy: 0.0000e+00
Epoch 213/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933385351168.0000 - accuracy: 0.0000e+00
Epoch 214/1000
1/1 [==============================] - 0s 1ms/step - loss: 1101933653786624.0000 - accuracy: 0.0000e+00
Epoch 215/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933653786624.0000 - accuracy: 0.0000e+00
Epoch 216/1000
1/1 [==============================] - 0s 0s/st

1/1 [==============================] - 0s 4ms/step - loss: 1101933318242304.0000 - accuracy: 0.0000e+00
Epoch 278/1000
1/1 [==============================] - 0s 8ms/step - loss: 1101933318242304.0000 - accuracy: 0.0000e+00
Epoch 279/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101933184024576.0000 - accuracy: 0.0000e+00
Epoch 280/1000
1/1 [==============================] - 0s 29ms/step - loss: 1101933385351168.0000 - accuracy: 0.0000e+00
Epoch 281/1000
1/1 [==============================] - 0s 6ms/step - loss: 1101933385351168.0000 - accuracy: 0.0000e+00
Epoch 282/1000
1/1 [==============================] - 0s 3ms/step - loss: 1101933184024576.0000 - accuracy: 0.0000e+00
Epoch 283/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933318242304.0000 - accuracy: 0.0000e+00
Epoch 284/1000
1/1 [==============================] - 0s 3ms/step - loss: 1101933318242304.0000 - accuracy: 0.0000e+00
Epoch 285/1000
1/1 [==============================] - 0s 4ms/st

1/1 [==============================] - 0s 3ms/step - loss: 1101932982697984.0000 - accuracy: 0.0000e+00
Epoch 347/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933049806848.0000 - accuracy: 0.0000e+00
Epoch 348/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933184024576.0000 - accuracy: 0.0000e+00
Epoch 349/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933049806848.0000 - accuracy: 0.0000e+00
Epoch 350/1000
1/1 [==============================] - 0s 0s/step - loss: 1101933184024576.0000 - accuracy: 0.0000e+00
Epoch 351/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101933184024576.0000 - accuracy: 0.0000e+00
Epoch 352/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101933049806848.0000 - accuracy: 0.0000e+00
Epoch 353/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101932982697984.0000 - accuracy: 0.0000e+00
Epoch 354/1000
1/1 [==============================] - 0s 16ms/st

1/1 [==============================] - 0s 4ms/step - loss: 1101933049806848.0000 - accuracy: 0.0000e+00
Epoch 416/1000
1/1 [==============================] - 0s 7ms/step - loss: 1101933184024576.0000 - accuracy: 0.0000e+00
Epoch 417/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101933049806848.0000 - accuracy: 0.0000e+00
Epoch 418/1000
1/1 [==============================] - 0s 0s/step - loss: 1101932848480256.0000 - accuracy: 0.0000e+00
Epoch 419/1000
1/1 [==============================] - 0s 0s/step - loss: 1101932982697984.0000 - accuracy: 0.0000e+00
Epoch 420/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101932982697984.0000 - accuracy: 0.0000e+00
Epoch 421/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101933049806848.0000 - accuracy: 0.0000e+00
Epoch 422/1000
1/1 [==============================] - 0s 8ms/step - loss: 1101932982697984.0000 - accuracy: 0.0000e+00
Epoch 423/1000
1/1 [==============================] - 0s 0s/ste

1/1 [==============================] - 0s 16ms/step - loss: 1101932848480256.0000 - accuracy: 0.0000e+00
Epoch 485/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101932781371392.0000 - accuracy: 0.0000e+00
Epoch 486/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101932781371392.0000 - accuracy: 0.0000e+00
Epoch 487/1000
1/1 [==============================] - 0s 34ms/step - loss: 1101932781371392.0000 - accuracy: 0.0000e+00
Epoch 488/1000
1/1 [==============================] - 0s 0s/step - loss: 1101932848480256.0000 - accuracy: 0.0000e+00
Epoch 489/1000
1/1 [==============================] - 0s 17ms/step - loss: 1101932781371392.0000 - accuracy: 0.0000e+00
Epoch 490/1000
1/1 [==============================] - 0s 0s/step - loss: 1101932512935936.0000 - accuracy: 0.0000e+00
Epoch 491/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101932781371392.0000 - accuracy: 0.0000e+00
Epoch 492/1000
1/1 [==============================] - 0s 4ms/

1/1 [==============================] - 0s 4ms/step - loss: 1101932512935936.0000 - accuracy: 0.0000e+00
Epoch 554/1000
1/1 [==============================] - 0s 16ms/step - loss: 1101932647153664.0000 - accuracy: 0.0000e+00
Epoch 555/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101932647153664.0000 - accuracy: 0.0000e+00
Epoch 556/1000
1/1 [==============================] - 0s 36ms/step - loss: 1101932647153664.0000 - accuracy: 0.0000e+00
Epoch 557/1000
1/1 [==============================] - 0s 4ms/step - loss: 1101932512935936.0000 - accuracy: 0.0000e+00
Epoch 558/1000
1/1 [==============================] - 0s 0s/step - loss: 1101932512935936.0000 - accuracy: 0.0000e+00
Epoch 559/1000
1/1 [==============================] - 0s 0s/step - loss: 1101932512935936.0000 - accuracy: 0.0000e+00
Epoch 560/1000
1/1 [==============================] - 0s 3ms/step - loss: 1101932512935936.0000 - accuracy: 0.0000e+00
Epoch 561/1000
1/1 [==============================] - 0s 4ms/st

### compare min loss

In [ ]:
models_min_loss, min_loss = mc.compare_min("loss")

In [ ]:
models_min_loss